## Tools

Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:
- a schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
- A function or coroutine to execute.

In [8]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
model=init_chat_model("groq:llama-3.3-70b-versatile")
response=model.invoke("Why do parrots talk?")
response.content

"Parrots are known for their ability to mimic human speech and other sounds, and they do so for several reasons. Here are some possible explanations:\n\n1. **Communication**: In the wild, parrots use vocalizations to communicate with other parrots, often to convey information about food, potential mates, or warning calls to alert others to predators. By mimicking human speech, parrots may be attempting to communicate with their human caregivers or other parrots in a way that is meaningful to them.\n2. **Social bonding**: Parrots are highly social animals that thrive on interaction and attention. By mimicking human speech, they may be seeking to bond with their human caregivers, much like they would with other parrots in the wild. Talking can be a way for parrots to initiate interaction, get attention, or even manipulate their owners into doing something they want.\n3. **Imitation and learning**: Parrots are renowned for their intelligence and ability to learn. They may mimic human spee

In [9]:
from langchain.tools import tool

@tool #we use this as a decorator, when we use this decorator on any function that function becomes a tool
def get_weather(location:str)->str:
    """Get the weather at a location""" #this is a doc string
    #whenever we tie a tool to the llm, the llm would be able to figure out the function of the tool using the doc string
    return f"It's sunny in {location}" #we have hardcoded this we can also use api's and get amswers from databases

#how to bind the LLM to the tool
model_with_tools=model.bind_tools([get_weather])
#or use the method used in model_integration.ipynb file

In [10]:
#how to call it
response=model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    #view tool calls made by the model
    print(f"Tool:{tool_call['name']}")
    print(f"Args:{tool_call['args']}")

content='' additional_kwargs={'tool_calls': [{'id': 'kenafkdps', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 220, 'total_tokens': 234, 'completion_time': 0.057473461, 'completion_tokens_details': None, 'prompt_time': 0.011197138, 'prompt_tokens_details': None, 'queue_time': 0.11917002, 'total_time': 0.068670599}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019c1e10-c714-7ae2-945e-485713624ddc-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'kenafkdps', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 220, 'output_tokens': 14, 'total_tokens': 234}
Tool:get_weather
Args:{'location': 'Boston'}


## Tool Execution Loops


In [11]:
#step 1: model generates tool calls
messages=[{"role":"user","content":"What's the weather in Boston?"}]
ai_msg=model_with_tools.invoke(messages)
messages.append(ai_msg)

#step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    tool_result=get_weather.invoke(tool_call) #Execute the tool with the generated arguments
    #whenever we make a tool call we are getting get_weather, we are doing this to get the response of this
    messages.append(tool_result)

#Step 3: Pass results back to model for final response
final_response=model_with_tools.invoke(messages)
print(final_response.text)


I'm glad you're interested in the weather! However, I should let you know that my previous response was an error on my part, as I didn't actually have the ability to provide real-time weather information. If you want to know the current weather in Boston, I can suggest checking a weather website or app for the most up-to-date information.
